In [ ]:
!pip3 install beautifulsoup4 requests selenium
!apt install chromium-chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 979 kB 29.6 MB/s 
     |████████████████████████████████| 359 kB 63.2 MB/s 
     |████████████████████████████████| 138 kB 60.5 MB/s 
     |████████████████████████████████| 138 kB 60.6 MB/s 
     |████████████████████████████████| 138 kB 58.0 MB/s 
     |████████████████████████████████| 138 kB 53.2 MB/s 
     |████████████████████████████████| 138 kB 55.2 MB/s 
     |████████████████████████████████| 153 kB 58.5 MB/s 
     |████████████████████████████████| 137 kB 6.1 MB/s 
     |████████████████████████████████| 136 kB 48.4 MB/s 
     |████████████████████████████████| 136 kB 60.1 MB/s 
     |████████████████████████████████| 136 kB 50.4 MB/s 
INFO: pip is looking at multiple versions of trio-websocket to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of attrs to d

In [ ]:
!pip install deeppavlov

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.18.0-cp37-cp37m-manylinux1_x86_64.whl (20.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.7 requires tensorflow>=2.0.0, which is not installed.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.21.2 which is incompatible.
xarray 0.20.2 requires pandas>=1.1, but you have pandas 0.25.3 which is incompatible.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.18.0 which is incompatible.
thinc 8.0.17 requires pydantic!=1.8,!=1.8.1,<1.9.0,>=1.7.4, but you have pydantic 1.3 which is incompatible.
tensorflow-probability 0.16.0 requires gast>=0.3.2

# Ria news data collecting

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import json
import pandas as pd
import numpy as np
import string

In [ ]:
def get_urls_by_category(start_url, category_name, urls_number = 500):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome('chromedriver', options=chrome_options)
    driver.switch_to.window(driver.window_handles[0])
    
    urls = []
    driver.get(start_url + category_name)
    contents = driver.find_elements_by_class_name("list-item__content")
    for content in contents:
        urls.append(content.find_element_by_css_selector('a').get_attribute('href'))
    next_page = driver.find_element_by_class_name("list-more")
    next_page_url = start_url + next_page.get_attribute("data-url")
    urls_count = len(urls)

    while(urls_count < urls_number):
        driver.get(next_page_url)
        contents = driver.find_elements_by_class_name("list-item__content")
        for content in contents:
            urls.append(content.find_element_by_css_selector('a').get_attribute('href'))
        urls_count += len(contents)
        next_page = driver.find_element_by_class_name("list-items-loaded")
        next_page_url = start_url + next_page.get_attribute("data-next-url")

    driver.close()
    return urls

In [ ]:
def write_urls():
    start_url = "https://ria.ru/"
    categories = ["politics", "economy", "science", "culture", "incidents"]
    urls = {}
    for category in categories:
        print("processing category ", category, '..\n')
        urls[category] = get_urls_by_category(start_url, category)
    with open('/content/drive/MyDrive/MAGOLEGO/dataset/urls_500.json', 'w') as f:
        json.dump(urls, f, ensure_ascii=False)

In [ ]:
write_urls()

processing category  politics ..

processing category  economy ..

processing category  science ..

processing category  culture ..

processing category  incidents ..



In [ ]:
ria_articles_path = '/content/drive/MyDrive/MAGOLEGO/dataset/ria_articles_500.json'

In [ ]:
with open('/content/drive/MyDrive/MAGOLEGO/dataset/urls.json', 'r') as f: 
    urls_dict = json.load(f) 
headers = requests.utils.default_headers()
headers.update({ 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Gecko/20100101 Safari/537.36'})
num_items_in_category = 500
num_items_counter = 0 
with open(ria_articles_path, 'a') as f:
    f.write("{\"catalog\": [\n")

for category, urls in urls_dict.items():
    print("processing category ", category, '..\n')
    num_items_counter = 0 
    for url in urls:
        try:
            page = requests.get(url)
            soup = BeautifulSoup(page.text, 'lxml')
            title = soup.find("meta",  {"name":"analytics:title"})["content"]
            tags = soup.find("meta",  {"name":"analytics:tags"})["content"]
            text = soup.find("div", itemprop="articleBody").text
            with open(ria_articles_path, 'a') as f:
                f.write(json.dumps({
                    "article_id": url,
                    "title": title,
                    "category": category,
                    "tags": tags,
                    "text": text
                }, ensure_ascii=False) + ',\n')
            num_items_counter += 1
        except:
            pass
        if (num_items_counter == num_items_in_category):
            break

with open(ria_articles_path, 'a') as f:
    f.truncate(f.tell()-2)
    f.write("\n]\n}")

processing category  politics ..

processing category  economy ..

processing category  science ..

processing category  culture ..

processing category  incidents ..



# Name entity recognition && linking in text news

In [ ]:
!pip install pybind11

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install hdt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install deeppavlov

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -m deeppavlov install entity_linking_rus

2022-06-21 19:41:31.915 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'entity_linking_rus' as '/usr/local/lib/python3.7/dist-packages/deeppavlov/configs/kbqa/entity_linking_rus.json'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-dwj3cikz
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-dwj3cikz
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached en_core_web_sm-2.2.5-py3-none-any.whl
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simp

In [ ]:
categories_dict = {'culture':0, 'economy':0, 'incidents':0, 'politics':0, 'science':0}

In [ ]:
articles_json = '/content/drive/MyDrive/MAGOLEGO/dataset/ria_articles_500.json'
with open(articles_json) as json_file:
    data = json.load(json_file)['catalog']
start_dataset = []

for article in data:
    category = article['category']
    text_len = len(article['text'])
    if categories_dict[category] > 49:
        categories_dict[category] += 1
        continue
    if text_len > 2000:
        continue
    categories_dict[category] += 1
    start_dataset.append([article['category'], 
                          article['text']])
df = pd.DataFrame(start_dataset,  columns =['category', 'text']) 
df

,category,text
0,politics,"МОСКВА, 27 мая — РИА Новости. Депутат Госдумы ..."
1,politics,"МОСКВА, 26 мая - РИА Новости. ""Западным друзья..."
2,politics,"МОСКВА, 26 мая/ Радио Sputnik. Канцлер Германи..."
3,politics,"МОСКВА, 26 мая - РИА Новости. У России есть пл..."
4,politics,"МОСКВА, 26 мая - РИА Новости. В основе работы ..."
...,...,...
245,incidents,"ЕКАТЕРИНБУРГ, 26 мая — РИА Новости. Свердловск..."
246,incidents,"МОСКВА, 26 мая/ Радио Sputnik. Житель города Ч..."
247,incidents,"МОСКВА, 26 мая/ Радио Sputnik. Взломавших сейф..."
248,incidents,"МОСКВА, 26 мая/ Радио Sputnik. Пьяный мужчина..."


In [ ]:
from deeppavlov import configs, build_model

In [ ]:
el_model = build_model(configs.kbqa.entity_linking_rus, download=True)

2022-06-21 19:43:30.973 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz download because of matching hashes
2022-06-21 19:43:33.580 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/kbqa/wikidata/q_to_descr_ru.pickle download because of matching hashes
2022-06-21 19:43:41.921 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/kbqa/models/ner_rus_bert.tar.gz download because of matching hashes
2022-06-21 19:44:00.471 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/kbqa/wikidata/entity_linking_rus.tar.gz download because of matching hashes
2022-06-21 19:44:05.339 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip download because of matching hashes
2022-06-21 19:44:08.90 INFO in











The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use standard file APIs to check for files with this prefix.


2022-06-21 19:44:45.936 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/entity_ranking_bert_rus_no_mention/model]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/entity_ranking_bert_rus_no_mention/model


2022-06-21 19:44:50.399 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_rus_bert/tag.dict]


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.



2022-06-21 19:45:09.342 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_rus_bert/model]


INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_rus_bert/model


In [ ]:
!pip install wikidata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from wikidata.client import Client

In [ ]:
client = Client() 
wiki_ids = []
wiki_names = []
human_label = 'Q5'
for i, row in df.iterrows():
    print(i, 'is proccesing')
    try:
        model_predict = el_model([df.iloc[i]['text']])
    except:
        wiki_ids.append([])
        wiki_names.append([])
        continue
    unique_ids = list(set(model_predict[2][0]))
    human_ids = []
    names = []
    for id in unique_ids:
        try:
            entity = client.get(id, load=True)
            label = entity.data['claims']['P31'][0]['mainsnak']['datavalue']['value']['id']
            name = entity.label.texts['ru'].translate(str.maketrans('', '', string.punctuation))
            if label == human_label:
                human_ids.append(id)
                names.append(name)
        except:
            continue
    wiki_ids.append(human_ids)
    wiki_names.append(names)

df['wiki_ids'] = wiki_ids
df['wiki_names'] = wiki_names

0 is proccesing
1 is proccesing
2 is proccesing
3 is proccesing
4 is proccesing
5 is proccesing
6 is proccesing
7 is proccesing
8 is proccesing
9 is proccesing
10 is proccesing
11 is proccesing
12 is proccesing
13 is proccesing
14 is proccesing
15 is proccesing
16 is proccesing
17 is proccesing
18 is proccesing
19 is proccesing
20 is proccesing
21 is proccesing
22 is proccesing
23 is proccesing
24 is proccesing
25 is proccesing
26 is proccesing
27 is proccesing
28 is proccesing
29 is proccesing
30 is proccesing
31 is proccesing
32 is proccesing
33 is proccesing
34 is proccesing
35 is proccesing
36 is proccesing
37 is proccesing
38 is proccesing
39 is proccesing
40 is proccesing
41 is proccesing
42 is proccesing
43 is proccesing
44 is proccesing
45 is proccesing
46 is proccesing
47 is proccesing
48 is proccesing
49 is proccesing
50 is proccesing
51 is proccesing
52 is proccesing
53 is proccesing
54 is proccesing
55 is proccesing
56 is proccesing
57 is proccesing
58 is proccesing
59 is p

In [ ]:
df.to_csv('/content/drive/MyDrive/MAGOLEGO/dataset/persons_wiki.csv')  

# Final Dataset

In [3]:
df = pd.read_csv('/content/drive/MyDrive/MAGOLEGO/dataset/persons_wiki.csv')

In [4]:
df

,Unnamed: 0,category,text,wiki_ids,wiki_names
0,0,politics,"МОСКВА, 27 мая — РИА Новости. Депутат Госдумы ...","['Q982461', 'Q4071023', 'Q4215364']","['Вячеслав Викторович Володин', 'Аршба Отари И..."
1,1,politics,"МОСКВА, 26 мая - РИА Новости. ""Западным друзья...",['Q58217'],['Сергей Викторович Лавров']
2,2,politics,"МОСКВА, 26 мая/ Радио Sputnik. Канцлер Германи...","['Q4384595', 'Q23530', 'Q117559', 'Q76658', 'Q...","['Владимир Андреевич Пучков', 'Дмитрий Анатоль..."
3,3,politics,"МОСКВА, 26 мая - РИА Новости. У России есть пл...","['Q57331', 'Q4252323', 'Q58217']","['Кабус бен Саид', 'Николай Павлович Лавёров',..."
4,4,politics,"МОСКВА, 26 мая - РИА Новости. В основе работы ...",['Q982461'],['Вячеслав Викторович Володин']
...,...,...,...,...,...
245,245,incidents,"ЕКАТЕРИНБУРГ, 26 мая — РИА Новости. Свердловск...",['Q4396754'],['Евгений Вадимович Ройзман']
246,246,incidents,"МОСКВА, 26 мая/ Радио Sputnik. Житель города Ч...",[],[]
247,247,incidents,"МОСКВА, 26 мая/ Радио Sputnik. Взломавших сейф...","['Q4241301', 'Q4497013', 'Q4067722']","['Владимир Евгеньевич Кристовский', 'Надежда Д..."
248,248,incidents,"МОСКВА, 26 мая/ Радио Sputnik. Пьяный мужчина...",[],[]
